# Description 

## Notes

- reward = level_player * level_food creating a clear incentive to cooperate
- learning to navigate the world is not needed. One could simplify use the action space: Of all available foods choose one where to go given its location, level and the position of the other players. Would make interpretation better as one could see which food was targeted. First progeam a basic rl algorithm (look into marllib). 

# Code

### Setup

#### Requirements

In [1]:
import lbforaging
import gymnasium as gym
from lbforaging.agents import H1
import time
import numpy as np

from tr_lbf_addon.lbf_gym import Lbf_Gym

#### Create the enviromente

In [17]:
# register the environment in gym

field_size = 8 # size of the game board
number_players = 2 # Number of players
max_num_food = 8 # max amount of food on the board. TODO How is teh amount of food determined?
coop_mode = False # If true, all foods will have teh max level so that all foods can only be loaded by working with other players
max_episode_steps = 50 # Number of steps until one round (episode) is terminated
sight = 0  #  How far can the agents see i presume TODO
max_player_level = 1
min_player_level = 1
max_food_level = 1
min_food_level = 1
normalize_reward = True
grid_observation = False
observe_agent_levels = True # If true, the observation will include the level of the agents
penalty = 0.0 # if the player was not the one to load the food, it gets a penalty to its reward
render_mode = "human"
full_info_mode = True

id_string = "Foraging-{0}x{0}-{1}p-{2}f{3}-v3".format(field_size, number_players, max_num_food, "-coop" if coop_mode else "")

gym.envs.registration.register(
    id=id_string,
    entry_point="lbforaging.foraging:ForagingEnv",
    kwargs={
        "players": number_players,
        "max_player_level": max_player_level,
        "min_player_level": min_player_level, 
        "max_food_level": max_food_level,
        "min_food_level": min_food_level,
        "field_size": (field_size, field_size),
        "max_num_food": max_num_food,
        "sight": sight,
        "max_episode_steps": max_episode_steps,
        "force_coop": coop_mode,
        "normalize_reward" : normalize_reward,
        "grid_observation" : grid_observation,
        "observe_agent_levels" : observe_agent_levels,
        "penalty" : penalty,
        "render_mode" : render_mode,
        "full_info_mode": full_info_mode
        },
    )

In [18]:
# define the environment. A more detailed way is discribed on https://github.com/semitable/lb-foraging
env = gym.make(id_string) # "Foraging-{GRID_SIZE}x{GRID_SIZE}-{PLAYER COUNT}p-{FOOD LOCATIONS}f{-coop IF COOPERATIVE MODE}-v0"
# render_mode is "human" per default

# reset the environment with a seed
observation, info = env.reset(seed=42)

# initialize the class
tr_marla_class = Lbf_Gym(observation[0])

### Tests

the goal now is to first replicate the tsimple heuristic, moving teh agent to to a given target. 
Then deploy a neural network that takes as its input the state of teh game and the position of the othe players and returns the fruit to go to. Output of the neural network is the fruit the agent should go to. Problem: How to label the fruit so that the neural network can learn over the course of several geames. Possibility. Output layer size = number of max fruit. Input is also the mapping giving each fruit in the current state a number. Then iterative: Go through each fruit and rate it with a nukber between 0 and 1. Later take the fruit with teh highest score. If two fruits have the same score choose randomely between them. Has the benefit that you can give summary statistics as input as well, which the neural network does not have to learn. So input is the position of the player, the game sioze, the position of the fruits, the distance between each fruit to another fruit and so on. This way teh neural network learns the correlation between these values and rates each fruit. Question: Does this acoount for cooperation well enough? Should. 

The astar pathfinding sometimes does not find a path for no reason todo

In [19]:
episode_over = False
step_amount = 0
while not episode_over:
    tr_marla_class.update_observation(observation[0])
    actions = tr_marla_class.agents_choose_actions()
    observation, reward, terminated, truncated, info = env.step(tuple(actions))
    # let 2 senconds pass
    env.render()
    time.sleep(0.2)
    episode_over = terminated or truncated
    step_amount += 1
env.close()
print(f"Game lasted {step_amount} steps")

Game lasted 13 steps


In [ ]:
tr_marla_class.update_observation(observation[0])
actions = tr_marla_class.agents_choose_actions()
observation, reward, terminated, truncated, info = env.step(tuple(actions))
tr_marla_class.full_info_field